<a href="https://colab.research.google.com/github/derek-shing/Forex2/blob/master/ZeroMQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.youtube.com/watch?v=3nM0c2kG_Sw&vl=en
